In [ ]:
#|hide
from nblite import nbl_export, show_doc; nbl_export();

In [ ]:
from netrun_sim import (
    # Graph types
    Graph, Node, Edge, Port, PortRef, PortType, PortSlotSpec,
    PortState, SalvoCondition, SalvoConditionTerm, MaxSalvos, Salvo,
    # NetSim types
    NetSim, NetAction, NetActionResponseData, PacketLocation,
)

In [ ]:
node_a = Node(
    name="A",
    out_ports={
        "out1": Port(PortSlotSpec.infinite()),
        "out2": Port(PortSlotSpec.finite(1)),
    },
    in_salvo_conditions={
        "manual": SalvoCondition(
            max_salvos=MaxSalvos.finite(1),
            ports=[],
            term=SalvoConditionTerm.true_(),
        ),
    },
    out_salvo_conditions={
        "1": SalvoCondition(
            max_salvos=MaxSalvos.finite(1),
            ports=["out1"],
            term=SalvoConditionTerm.port("out1", PortState.non_empty()),
        ),
        "2": SalvoCondition(
            max_salvos=MaxSalvos.finite(1),
            ports=["out2"],
            term=SalvoConditionTerm.port("out2", PortState.non_empty()),
        ),
    },
)

In [ ]:
node_b1 = Node(
    name="B1",
    in_ports={"in": Port(PortSlotSpec.infinite())},
    out_ports={"out": Port(PortSlotSpec.infinite())},
    in_salvo_conditions={
        "default": SalvoCondition(
            max_salvos=MaxSalvos.finite(1),
            ports=["in"],
            term=SalvoConditionTerm.port("in", PortState.non_empty()),
        ),
    },
    out_salvo_conditions={
        "default": SalvoCondition(
            max_salvos=MaxSalvos.Infinite,
            ports=["out"],
            term=SalvoConditionTerm.port("out", PortState.non_empty()),
        ),
    },
)

In [ ]:
node_b2 = Node(
    name="B2",
    in_ports={"in": Port(PortSlotSpec.infinite())},
    out_ports={"out": Port(PortSlotSpec.infinite())},
    in_salvo_conditions={
        "default": SalvoCondition(
            max_salvos=MaxSalvos.finite(1),
            ports=["in"],
            term=SalvoConditionTerm.port("in", PortState.non_empty()),
        ),
    },
    out_salvo_conditions={
        "default": SalvoCondition(
            max_salvos=MaxSalvos.Infinite,
            ports=["out"],
            term=SalvoConditionTerm.port("out", PortState.non_empty()),
        ),
    },
)

In [ ]:
node_c = Node(
    name="C",
    in_ports={"in1": Port(PortSlotSpec.infinite()), "in2": Port(PortSlotSpec.infinite())},
    out_ports={"out": Port(PortSlotSpec.infinite())},
)

In [ ]:
edges = [
    Edge(
        PortRef("A", PortType.Output, "out1"),
        PortRef("B1", PortType.Input, "in"),
    ),
    Edge(
        PortRef("A", PortType.Output, "out2"),
        PortRef("B2", PortType.Input, "in"),
    ),
    Edge(
        PortRef("B1", PortType.Output, "out"),
        PortRef("C", PortType.Input, "in1"),
    ),
    Edge(
        PortRef("B2", PortType.Output, "out"),
        PortRef("C", PortType.Input, "in2"),
    ),
]

In [ ]:
graph = Graph([node_a, node_b1, node_b2, node_c], edges)
graph.validate()

[]

In [ ]:
net = NetSim(graph)

In [ ]:
response_data, events = net.do_action(
    NetAction.run_net_until_blocked()
)
assert events == []

In [ ]:
response_data, events = net.do_action(NetAction.create_epoch(
    "A",
    Salvo(
        "manual",
        []
    )
))

response_data, events

(<NetActionResponseData_CreatedEpoch at 0x111b259a0>,
 [NetEvent.EpochCreated(ts=1768318716455, epoch_id='01KEW04JH7E7F8B0WX7P6JQ2X2')])

In [ ]:
epoch_id = response_data.epoch.id
response_data, events = net.do_action(NetAction.start_epoch(
    epoch_id
))
response_data, events

(<NetActionResponseData_StartedEpoch at 0x111b25bb0>,
 [NetEvent.EpochStarted(ts=1768318716463, epoch_id='01KEW04JH7E7F8B0WX7P6JQ2X2')])

In [ ]:
response_data, events = net.do_action(NetAction.create_packet(
    epoch_id,
))
packetA1_id = response_data.packet_id
response_data, events

(<NetActionResponseData_Packet at 0x111b25d10>,
 [NetEvent.PacketCreated(ts=1768318716468, packet_id='01KEW04JHMJCYG5Z50F9X6CD08')])

In [ ]:
response_data, events = net.do_action(NetAction.create_packet(
    epoch_id,
))
packetA2_id = response_data.packet_id
response_data, events

(<NetActionResponseData_Packet at 0x111b25c60>,
 [NetEvent.PacketCreated(ts=1768318716472, packet_id='01KEW04JHRGEZKAAB26Y1DDJPW')])

In [ ]:
response_data, events = net.do_action(NetAction.load_packet_into_output_port(
    packetA1_id,
    "out1"
))

response_data, events = net.do_action(NetAction.send_output_salvo(
    epoch_id,
    "1"
))

response_data, events = net.do_action(NetAction.load_packet_into_output_port(
    packetA2_id,
    "out2"
))

response_data, events = net.do_action(NetAction.send_output_salvo(
    epoch_id,
    "2"
))

OutputPortFullError: Output port 'out2' is full in epoch 01KEW04JH7E7F8B0WX7P6JQ2X2